# Modelling in Python
This project aims to an overview of the most significant modelling tools using python's libraries.

### Import external libraries

In [1]:
# Basics
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Charts
import plotly.express as px
import plotly.graph_objects as go
from patsy import dmatrices

# Anova test
from statsmodels.stats.anova import anova_lm


## Preliminar Analysis

In [2]:
df = pd.read_csv("../Datasets/cars.csv")

In [3]:
df.head()

,MPG,cylind,disp,HP,weight,seconds,year,origin,name
0,18.0,8,3.07,1.30,3.504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,3.50,1.65,3.693,11.5,70,1,buick skylark 320
2,18.0,8,3.18,1.50,3.436,11.0,70,1,plymouth satellite
3,16.0,8,3.04,1.50,3.433,12.0,70,1,amc rebel sst
4,17.0,8,3.02,1.40,3.449,10.5,70,1,ford torino


In [4]:
df.isna().sum()

MPG        0
cylind     0
disp       0
HP         0
weight     0
seconds    0
year       0
origin     0
name       0
dtype: int64

In [5]:
df.dtypes

MPG        float64
cylind       int64
disp       float64
HP         float64
weight     float64
seconds    float64
year         int64
origin       int64
name        object
dtype: object

In [6]:
fig = px.histogram(df.MPG, nbins=20, title="Histogram of Frequency")
fig.show()

In [7]:
fig = px.scatter(df, df.weight, df.MPG, title="Scatter Plot")
fig.show()

In [8]:
residuals_dfdf = df.sort_values(by='year').reset_index(drop=True).copy()

In [9]:
fig = px.box(df, x='year', y='MPG', title='Box Plot')
fig.show()

In [10]:
fig = px.box(df, x='origin', y='MPG', title='Box Plot')
fig.show()

## Models

In [11]:
Y = df.MPG
X = sm.add_constant(df.weight)
m0 = sm.OLS(Y, X).fit()
print(m0.summary())

                            OLS Regression Results                            
Dep. Variable:                    MPG   R-squared:                       0.693
Model:                            OLS   Adj. R-squared:                  0.692
Method:                 Least Squares   F-statistic:                     878.8
Date:                Mon, 14 Aug 2023   Prob (F-statistic):          6.02e-102
Time:                        10:59:46   Log-Likelihood:                -1130.0
No. Observations:                 392   AIC:                             2264.
Df Residuals:                     390   BIC:                             2272.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         46.2165      0.799     57.867      0.0

- intercept: average consumption of a machine weighing 0 (x = 0)
- gradient: 7 miles per gallon are lost per 1000lb
- R squared: about 70% of the variability of y.
- Statistics F: null hypothesis that the beta weight parameter = 0
- Stat F on one parameter = statistics T squared
- Residual Standard error: sum of squared residues / N - k, estimate of the standard deviation of the error component

In [12]:
fig = go.Figure(data=go.Scatter(x=list(range(len(m0.resid))), y=m0.resid, mode='lines'))
fig.update_layout(title='Residuals of M0', xaxis_title='Observations', yaxis_title='Residuals')
fig.add_shape(type="line", x0=0, x1=len(m0.resid), y0=0, y1=0, line=dict(color="black", dash="dash"))
fig.show()

In [13]:
residuals_df = pd.DataFrame({'residuals': m0.resid, 'year': df['year']})
fig = px.box(residuals_df, residuals_df.year, residuals_df.residuals, title='Box Plot')
fig.show()

Not-null Mean within each year, we are detached from the exogenity hypothesis because the error term must have average = 0 against the values of x, if there is correlation between year and weight then there is no exogenity since the variable weight is not present in the current model. The model first overestimates and then underestimates.

### Comparing two Nested Models

In [14]:
Y = df.MPG
X = sm.add_constant(df[['year', 'weight']])
m1 = sm.OLS(Y, X).fit()
print(m1.summary())

                            OLS Regression Results                            
Dep. Variable:                    MPG   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.807
Method:                 Least Squares   F-statistic:                     819.5
Date:                Mon, 14 Aug 2023   Prob (F-statistic):          3.33e-140
Time:                        10:59:46   Log-Likelihood:                -1037.6
No. Observations:                 392   AIC:                             2081.
Df Residuals:                     389   BIC:                             2093.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -14.3473      4.007     -3.581      0.0

In [15]:
Y = df.MPG
X = sm.add_constant(df['weight'])
m2 = sm.OLS(Y, X.join(pd.get_dummies(df['year'], prefix='year'))).fit()
print(m2.summary())

                            OLS Regression Results                            
Dep. Variable:                    MPG   R-squared:                       0.842
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     155.0
Date:                Mon, 14 Aug 2023   Prob (F-statistic):          1.68e-142
Time:                        10:59:47   Log-Likelihood:                -999.51
No. Observations:                 392   AIC:                             2027.
Df Residuals:                     378   BIC:                             2083.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         39.4655      0.576     68.484      0.0

Thanks to the model with the variable 'year' it is clear that there were endogenity in the past one: the weight params change unveiling the distortion, indeed a part of the effect of the variable 'time' was attributed to 'weight'. The R squared increases.

M1 assumes that efficiency is linear function of time, constant marginal effect for each year while M2 assumes that each year may have different effects, and here the intercept is the efficiency that a machine with weight 0 has in year 0 (base year). Since a restriction can be imposed on all parameters of M2 such that ßi = i * ß1 where ß1 is the first parameter, M2 and M1 are nested models. Precisely because one can be written as a restricted model of the other, and then you can use ANOVA Test F to estimate which is the most efficient.

In [16]:
anova_results = anova_lm(m1, m2)
print(anova_results)

   df_resid          ssr  df_diff     ss_diff         F        Pr(>F)
0     389.0  4568.952042      0.0         NaN       NaN           NaN
1     378.0  3762.754586     11.0  806.197455  7.362658  1.896466e-11


The decrease in RSS is statistically significant, therefore the second model is preferred (with dummy variables per year)

### Model with more Explanatory Variables

In [17]:
Y = df.MPG
X = sm.add_constant(df[['weight', 'disp', 'HP', 'seconds']])
m3 = sm.OLS(Y, X.join(pd.get_dummies(df['year'], prefix='year'))).fit()
print(m3.summary())

                            OLS Regression Results                            
Dep. Variable:                    MPG   R-squared:                       0.843
Model:                            OLS   Adj. R-squared:                  0.836
Method:                 Least Squares   F-statistic:                     125.7
Date:                Mon, 14 Aug 2023   Prob (F-statistic):          1.10e-139
Time:                        10:59:48   Log-Likelihood:                -998.43
No. Observations:                 392   AIC:                             2031.
Df Residuals:                     375   BIC:                             2098.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         38.5840      1.732     22.276      0.0

Not being nested models, for the comparison between m3 and m1, we look at both the square R: m2 = 0.842, m3 = 0.843, and the efficiency and thrift of the model, in this case we prefer a model with fewer parameters than one with more parameters.

### Model with Interactions between Variables with Patsy

In [33]:
y, X = dmatrices('MPG ~ weight + C(origin) * year', data=df, return_type='dataframe')
m4 = sm.OLS(y, X).fit()
print(m4.summary())

                            OLS Regression Results                            
Dep. Variable:                    MPG   R-squared:                       0.829
Model:                            OLS   Adj. R-squared:                  0.826
Method:                 Least Squares   F-statistic:                     310.2
Date:                Mon, 14 Aug 2023   Prob (F-statistic):          4.59e-144
Time:                        11:07:09   Log-Likelihood:                -1015.5
No. Observations:                 392   AIC:                             2045.
Df Residuals:                     385   BIC:                             2073.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -3.7479    

Considering that the year 0 is 1900:
- intercept: efficiency if weight 0 and in 1900 and in America
- 0.59: effect of time on efficiency for American machines (every year gains...)
- weight: effect on efficiency equal for all countries
- factor(origin): different intercept for each country
European machines (origin = 2) will gain the sum of the previous values, then 0.59 + 0.53

In [34]:
y, X = dmatrices('MPG ~ weight + C(origin) + year', data=df, return_type='dataframe')
m5 = sm.OLS(y, X).fit()
print(m5.summary())

                            OLS Regression Results                            
Dep. Variable:                    MPG   R-squared:                       0.819
Model:                            OLS   Adj. R-squared:                  0.817
Method:                 Least Squares   F-statistic:                     437.9
Date:                Mon, 14 Aug 2023   Prob (F-statistic):          3.53e-142
Time:                        11:07:15   Log-Likelihood:                -1026.1
No. Observations:                 392   AIC:                             2062.
Df Residuals:                     387   BIC:                             2082.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        -18.3069      4.017     -4.

In [35]:
anova_results = anova_lm(m5, m4)
print(anova_results)

   df_resid          ssr  df_diff     ss_diff          F    Pr(>F)
0     387.0  4310.409498      0.0         NaN        NaN       NaN
1     385.0  4082.718827      2.0  227.690672  10.735604  0.000029


Here the P value is very low, highlighting the statistical significance of the combined effect of year and origin.

### Model with Intercept at 1970

In [36]:
df['year_minus_70'] = df['year'] - 70
y, X = dmatrices('MPG ~ weight + C(origin) + year_minus_70', data=df, return_type='dataframe')
m6 = sm.OLS(y, X).fit()
print(m6.summary())

                            OLS Regression Results                            
Dep. Variable:                    MPG   R-squared:                       0.819
Model:                            OLS   Adj. R-squared:                  0.817
Method:                 Least Squares   F-statistic:                     437.9
Date:                Mon, 14 Aug 2023   Prob (F-statistic):          3.53e-142
Time:                        11:07:29   Log-Likelihood:                -1026.1
No. Observations:                 392   AIC:                             2062.
Df Residuals:                     387   BIC:                             2082.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         35.5825      1.010     35.

### Model with non-linear Interactions

In [37]:
df['w2'] = df['weight']**2
y, X = dmatrices('MPG ~ weight + w2', data=df, return_type='dataframe')
m7 = sm.OLS(y, X).fit()
print(m7.summary())

                            OLS Regression Results                            
Dep. Variable:                    MPG   R-squared:                       0.715
Model:                            OLS   Adj. R-squared:                  0.714
Method:                 Least Squares   F-statistic:                     488.3
Date:                Mon, 14 Aug 2023   Prob (F-statistic):          8.39e-107
Time:                        11:07:35   Log-Likelihood:                -1115.1
No. Observations:                 392   AIC:                             2236.
Df Residuals:                     389   BIC:                             2248.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     62.2555      2.993     20.800      0.0

### Piecewise Linear Model

In [23]:
df['d3'] = 0
df['d3'][df['weight'] > 3] = 1

/var/folders/_s/3l81jppx5y5bjk_zkh9m6bdw0000gn/T/ipykernel_2653/1278950164.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [38]:
y, X = dmatrices('MPG ~ weight * d3', data=df, return_type='dataframe')
m8 = sm.OLS(y, X).fit()
print(m8.summary())

                            OLS Regression Results                            
Dep. Variable:                    MPG   R-squared:                       0.714
Model:                            OLS   Adj. R-squared:                  0.711
Method:                 Least Squares   F-statistic:                     322.3
Date:                Mon, 14 Aug 2023   Prob (F-statistic):          5.80e-105
Time:                        11:07:41   Log-Likelihood:                -1116.1
No. Observations:                 392   AIC:                             2240.
Df Residuals:                     388   BIC:                             2256.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     53.0450      1.930     27.481      0.0

The slope after the threshold is equal to the sum of base weight + weight:d3

In [25]:
anova_results = anova_lm(m0, m9)
print(anova_results)

   df_resid          ssr  df_diff     ss_diff          F    Pr(>F)
0     390.0  7321.233706      0.0         NaN        NaN       NaN
1     388.0  6820.637470      2.0  500.596236  14.238503  0.000001


A statistically significant p-value of both the d3 variable and the F test in relation to the model without dummy variable shows that the dummy variable has a significant effect on the variable being studied.

### Spline Model

In [39]:
df['w3'] = (df['weight'] - 3) * df['d3']
y, X = dmatrices('MPG ~ weight + w3', data=df, return_type='dataframe')
m9 = sm.OLS(y, X).fit()
print(m9.summary())

                            OLS Regression Results                            
Dep. Variable:                    MPG   R-squared:                       0.713
Model:                            OLS   Adj. R-squared:                  0.712
Method:                 Least Squares   F-statistic:                     484.3
Date:                Mon, 14 Aug 2023   Prob (F-statistic):          2.62e-106
Time:                        11:07:46   Log-Likelihood:                -1116.2
No. Observations:                 392   AIC:                             2238.
Df Residuals:                     389   BIC:                             2250.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     53.5805      1.585     33.802      0.0

The model contains an assumption in addition to the linear one at times and therefore less flexible, in fact more a model contains or is based on assumptions and less flexible but tendentially more robust in the specific cases of study.

## Summary

In [46]:
num_models = 9
models_df = pd.DataFrame(columns=['Name', 'R-squared'])

for i in range(0, num_models):
    model_name = f'm{i}'
    model = globals()[model_name]

    new_row = {"Name": model_name, "R-squared": model.rsquared}
    models_df.loc[i] = new_row

models_df


,Name,R-squared
0,m0,0.692630
1,m1,0.808180
2,m2,0.842027
3,m3,0.842893
4,m4,0.828594
5,m5,0.819035
6,m6,0.819035
7,m7,0.715148
8,m8,0.713647


### Thanks for Reading!